In [ ]:
import eodslib
from datetime import datetime
from pathlib import Path
from osgeo import gdal

In [ ]:
# user connection settings
conn = {
    'domain': 'https://earthobs.defra.gov.uk',
    'username': '<insert-user-name>',
    'access_token': '<insert-user-token>',
    }

In [ ]:
# user defined keyword arguments (kwargs)
output_dir = Path('/some/dir')
cat_search_geom_wgs84wkt_query = 'POLYGON((-1.64 53.38, -1.64 52.98, -1.17 52.98, -1.17 53.38, -1.64 53.38))'

In [ ]:
# example of keyword argument (kwarg) expansion, the input needs to be a python dictionary

eods_params = {
    'output_dir':output_dir,
    'title':'ORB137',
    'cloud_min':0,
    'cloud_max':15,
    'sat_id':2,
    'geom':cat_search_geom_wgs84wkt_query,
    'date_start':'2020-01-01',
    'date_end':'2020-09-01',
    'find_least_cloud':True,
    'ignore_split_granules':True    
    }

list_of_layers, df = eodslib.query_catalog(conn, **eods_params)

In [ ]:
# user can override 'wps job' config

config_wpsprocess = []

for lyr in list_of_layers:
       
    config_wpsprocess.append({
        'template_xml':'gsdownload_template.xml',
        'xml_config':{
             'template_layer_name':lyr,
             'template_outputformat':'image/tiff',
             'template_mimetype':'application/zip'},
        'dl_bool':True
    })

config_wpsprocess

In [ ]:
# run wps process with updated list of layers
list_download_paths = eodslib.run_wps(config_general, config_wpsprocess, output_dir=output_dir)

In [ ]:
# POST-PROCESSING EXAMPLE
#        convert gsdownload s2 output to an "optimised" GeoTIFF
#        build a gdal virtual file (VRT) (a mosaic)

# set gdal config
translateoptions = gdal.TranslateOptions(gdal.ParseCommandLine("-of Gtiff -co COMPRESS=LZW  -co BIGTIFF=YES -co TILED=YES -co BLOCKXSIZE=512 -co BLOCKYSIZE=512"))
gdal.SetConfigOption("COMPRESS_OVERVIEW", "DEFLATE")

outfile_list = []

# loop through all S2 output in output directory
for input_file in output_dir.glob('**/*osgb_vmsk_sharp_rad_srefdem_stdsref.tif'):
    
    print('\n' + datetime.utcnow().isoformat() + ' processing input :: ' + str(input_file))
    
    output_file = input_file.parent / str(input_file.stem + '-cog.tif')
    outfile_list.append(str(output_file))
    
    # run downloaded file via gdal translate
    gdal.Translate(str(output_file), str(input_file), options=translateoptions)
    
    # add overviews to translated file
    Image = gdal.Open(str(output_file), 1)    
    Image.BuildOverviews("NEAREST", [2,4,8,16,32,64, 128, 256, 512])
    
    print('\n' + datetime.utcnow().isoformat() + ' output here :: ' + str(output_file))
    
# write out vrt input file    
with open(output_dir / 'vrt-inputs.txt', 'w') as outtxt:
    for ii in outfile_list:
        outtxt.write(ii + '\n')

# run gdal build vrt from commandline
! gdalbuildvrt -resolution average -r nearest -input_file_list {str(output_dir / 'vrt-inputs.txt')} {str(output_dir / 'mosaic.vrt')}

In [ ]:
print('### Script Finished')